In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from dotenv import load_dotenv
import os


load_dotenv()
sender_email = os.getenv("SENDER_EMAIL")
sender_password = os.getenv("SENDER_PASSWORD")


your_text = "Run Travis"
pr_link = "https://github.com/apache/flink/pull/9900"
receiver_email = "huisun.uf@gmail.com"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.load_state_dict(torch.load('finetuned_BERT_epoch_5.model', map_location=device))
model.to(device)
model.eval()


encoded = tokenizer.encode_plus(
    your_text,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

input_ids = encoded['input_ids'].to(device)
attention_mask = encoded['attention_mask'].to(device)

with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=attention_mask)

logits = output.logits
predicted_class_id = torch.argmax(logits, dim=1).item()


subject = "[GitHub][Important] You have received a comment in a PR, the comment is important." \
    if predicted_class_id == 1 else \
    "[GitHub][Ignore] You have received a comment in a PR, the comment can be ignore."

body = f"""Hi there,

You have received a new comment in a GitHub pull request:

"{your_text}"

Pull Request Link: {pr_link}

Regards,
Your Friendly AI Reviewer 🤖
"""


msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

try:
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)
    print(f"✅ Email sent to {receiver_email} with subject:\n{subject}")
except Exception as e:
    print("❌ Failed to send email:", e)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\huisu\AppData\Local\Temp\ipykernel_14892\3122763918.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seriali

✅ Email sent to huisun.uf@gmail.com with subject:
[GitHub] You have received a comment in a PR, the comment can be ignore.
